# Lemmatize Forum Text
Cleans the data for topic modelling

## Data Sources
- youbemom-merged.db (scraped with 1-Scrape_Forum.ipynb)


## Changes
- 2020-09-14: Created
- 2020-09-15: Added functions for accessing database, cleaning/tokenizing text
- 2020-09-16: Generated and saved corpus and dictionary
- 2020-12-13: Updated for new data

## TODO
- Tutorial: https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21
- Add actual database structure

## Imports

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
from io import FileIO
import re
import string
# tokenizing
from nltk.corpus import wordnet as wn
from nltk.tokenize import RegexpTokenizer #, word_tokenize
from nltk.corpus import stopwords
# saving the corpus and dictionary
from gensim import corpora, models
import pickle
from youbemom import create_connection
import swifter

Prerequisite:

In [2]:
# import nltk
# nltk.download('wordnet')
# nltk.download('stopwords')

## Functions

For formatting the data

In [3]:
def clean_text(text):
    """ cleans the input text of punctuation, extra
        spaces, and makes letters lower case
    :param text: text (= title + body here)
    :return clean: clean text
    """
    clean = "".join([t for t in text if t not in string.punctuation])
    clean = re.sub(" +", " ", clean)
    clean = clean.strip()
    clean = clean.lower()
    return clean

In [4]:
def remove_stopwords(text):
    """ remove all stop words from the text
        using stopwords from nltk.corpus
    :param text: text with stopwords
    :return words: text without stopwords
    """
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [5]:
def encode_utf8(text):
    words = [w.encode() for w in text]
    return words

In [6]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [7]:
def lemmatize(text):
    lemmas = [get_lemma(w) for w in text]
    return lemmas

## File Locations

In [8]:
p = Path.cwd()
path_parent = p.parents[0]

In [9]:
path_db = path_parent / "database" / "youbemom-merged.db"
path_db = str(path_db)
path_lemma_pkl = path_parent / "clean_data" / "lemmatized_text_{}.pkl"
path_lemma_pkl = str(path_lemma_pkl)
path_corpus_pkl = path_parent / "clean_data" / "corpus_{}.pkl"
path_corpus_pkl = str(path_corpus_pkl)
path_dictionary_gensim = path_parent / "clean_data" / "dictionary_{}.gensim"
path_dictionary_gensim = str(path_dictionary_gensim)

## Load Data

In [10]:
sn_sql = ''' SELECT text_no_url FROM posts WHERE subforum="special-needs" '''
td_sql = ''' SELECT text_no_url FROM posts WHERE subforum="toddler" '''

In [11]:
sn_parents_sql = ''' SELECT text_no_url FROM posts WHERE subforum="special-needs" AND parent_id="" '''
td_parents_sql = ''' SELECT text_no_url FROM posts WHERE subforum="toddler" AND parent_id="" '''

In [12]:
conn = create_connection(path_db)

In [ ]:
sn = pd.read_sql_query(sn_sql, conn)

In [ ]:
td = pd.read_sql_query(td_sql, conn)

In [13]:
sn_parents = pd.read_sql_query(sn_parents_sql, conn)

In [14]:
td_parents = pd.read_sql_query(td_parents_sql, conn)

In [15]:
conn.commit()
conn.close()

## Tokenize/Lemmatize Text
Tokenize the data, removing stopwords, punctuation, and making all lower case. Then lemmatize the words. Create a corpus and dictionary, and save them.

### Special Needs

In [ ]:
text = sn['text_no_url']
text = text.swifter.apply(clean_text)
tokenizer = RegexpTokenizer(r'\w+')
text = text.swifter.apply(tokenizer.tokenize)
text = text.swifter.apply(remove_stopwords)
text = text.swifter.apply(lemmatize)
pickle.dump(text, open(path_lemma_pkl.format("special-needs"), 'wb'))

In [ ]:
dictionary = corpora.Dictionary(text)
dictionary.save(FileIO(path_dictionary_gensim.format("special-needs"), "wb"))

In [ ]:
corpus = [dictionary.doc2bow(t) for t in text]
pickle.dump(corpus, open(path_corpus_pkl.format("special-needs"), 'wb'))

In [18]:
text = sn_parents['text_no_url']
text = text.swifter.apply(clean_text)
tokenizer = RegexpTokenizer(r'\w+')
text = text.swifter.apply(tokenizer.tokenize)
text = text.swifter.apply(remove_stopwords)
text = text.swifter.apply(lemmatize)
pickle.dump(text, open(path_lemma_pkl.format("special-needs_parents"), 'wb'))

In [19]:
dictionary = corpora.Dictionary(text)
dictionary.save(FileIO(path_dictionary_gensim.format("special-needs_parents"), "wb"))

In [20]:
corpus = [dictionary.doc2bow(t) for t in text]
pickle.dump(corpus, open(path_corpus_pkl.format("special-needs_parents"), 'wb'))

### Toddler

In [ ]:
text = td['text_no_url']
text = text.swifter.apply(clean_text)
tokenizer = RegexpTokenizer(r'\w+')
text = text.swifter.apply(tokenizer.tokenize)
text = text.swifter.apply(remove_stopwords)
text = text.swifter.apply(lemmatize)
pickle.dump(text, open(path_lemma_pkl.format("toddler"), 'wb'))

In [ ]:
dictionary = corpora.Dictionary(text)
dictionary.save(FileIO(path_dictionary_gensim.format("toddler"), "wb"))

In [ ]:
corpus = [dictionary.doc2bow(t) for t in text]
pickle.dump(corpus, open(path_corpus_pkl.format("toddler"), 'wb'))

In [21]:
text = td_parents['text_no_url']
text = text.swifter.apply(clean_text)
tokenizer = RegexpTokenizer(r'\w+')
text = text.swifter.apply(tokenizer.tokenize)
text = text.swifter.apply(remove_stopwords)
text = text.swifter.apply(lemmatize)
pickle.dump(text, open(path_lemma_pkl.format("toddler_parents"), 'wb'))

In [22]:
dictionary = corpora.Dictionary(text)
dictionary.save(FileIO(path_dictionary_gensim.format("toddler_parents"), "wb"))

In [23]:
corpus = [dictionary.doc2bow(t) for t in text]
pickle.dump(corpus, open(path_corpus_pkl.format("toddler_parents"), 'wb'))